In [1]:
%matplotlib inline
%load_ext autoreload

%autoreload 2

In [2]:
import pandas as pd
from Bio import SeqIO
from matplotlib import pyplot as plt
import numpy as np

from collections import defaultdict, Counter

In [3]:
df = pd.read_csv('../Data/examples_for_testing/ecoli.gff3', sep='\t', skiprows=2, header=None)
df.columns = ["genome_id", "source", "type", "start", "stop", "idk", "strand", "trash", "qualifiers"]
df = df[df["type"]=='CDS']

In [ ]:
df.iloc[4]

In [ ]:

df

In [ ]:
df[6]

In [4]:
genome = list(SeqIO.parse('../Data/examples_for_testing/ecoli.fasta', "fasta"))[0]
#print(genome)

print(genome)


ID: NC_000913.3
Name: NC_000913.3
Description: NC_000913.3 Escherichia coli str. K-12 substr. MG1655, complete genome
Number of features: 0
Seq('AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAG...TTC', SingleLetterAlphabet())


In [ ]:
genome[0:10]

In [ ]:
genome_seq = list()
for genome in SeqIO.parse('../Data/examples_for_testing/ecoli.fasta', "fasta"):
    genome_seq.append(genome.seq)


genome = str(genome_seq[0])
print(genome)

In [ ]:
# ecoli_seq = SeqIO.index("../Data/examples_for_testing/ecoli.fasta", "fasta")

# full_seq = ecoli_seq.get_raw("NC_000913.3").decode()[72:]

In [ ]:
print(full_seq)

In [9]:
set(["a","b","c","a"])

{'a', 'b', 'c'}

In [10]:
len(set(df["genome_id"]))

1

In [15]:
assert(df.iloc[0]["genome_id"] == genome.id)

In [ ]:
df.head()

In [ ]:
# for i in df.iterrows():
#     print(df[1])
start_codon = []

for index in df.index:
    
    if df.loc[index]["strand"] == "+":
        temp_start = df.loc[index]["start"]
        #temp_stop = df.iloc[index]["stop"]
        start_seq = genome.seq[temp_start-1: temp_start+2]
    
    elif df.loc[index]["strand"] == "-":
        temp_start = df.loc[index]["stop"]
        #temp_stop = df.iloc[index]["start"]
        
        start_seq = genome.seq[temp_start - 3 : temp_start]
        start_seq = str(start_seq.reverse_complement())
    
    start_codon.append(str(start_seq))   
    
    
Counter(start_codon)

In [ ]:
neg_strand = []

for index in df.head(100).index:
    
    if df.loc[index]["strand"] == "-":
        temp_start_2 = df.loc[index]["stop"]
        #temp_stop_2 = df.iloc[index]["start"]
    
        neg_start_seq = genome.seq[temp_start_2 - 3 : temp_start_2]
    
        neg_strand.append(str(neg_start_seq.reverse_complement()))
        
Counter(neg_strand)

In [ ]:
upstream = []
upstream_len = 11


for index in df.head(100).index:
    
    if df.loc[index]["strand"] == "+":
        temp_start = df.loc[index]["start"]
        #temp_stop = df.iloc[index]["stop"]
        upstream_seq = genome.seq[temp_start - upstream_len: temp_start + 2]
    
    elif df.loc[index]["strand"] == "-":
        temp_start = df.loc[index]["stop"]
        #temp_stop = df.iloc[index]["start"]
        
        upstream_seq = genome.seq[temp_start - 3 : temp_start + 10]
        upstream_seq = str(upstream_seq.reverse_complement())
    
    upstream.append(str(upstream_seq))
    
print(upstream)
    

In [ ]:
start_loc = str(df[3].astype(int))


print(start_loc)

# Compiling useful bits of notebook

In [ ]:
#setting up dataframe
df = pd.read_csv('../Data/examples_for_testing/ecoli.gff3', sep='\t', skiprows=2, header=None)
df.columns = ["genome_id", "source", "type", "start", "stop", "idk", "strand", "trash", "qualifiers"]
df = df[df["type"]=='CDS']


#read in genome 
genome = list(SeqIO.parse('../Data/examples_for_testing/ecoli.fasta', "fasta"))[0]


#getting coding sequences
upstream_len = 20

for index in df.index:
    #Getting coding sequence
    temp_start = df.loc[index]["start"]
    temp_stop = df.loc[index]["stop"]
    coding_seq = genome.seq[temp_start-1: temp_stop]
        
    if df.loc[index]["strand"] == "-":
        coding_seq = coding_seq.reverse_complement()
        
        
    coding_seq = str(coding_seq)
    
    df.at[index, "coding_sequence"] = coding_seq
    
    #getting upstream seequence
    
    if df.loc[index]["strand"] == "+":
        
        upstream_seq = genome.seq[temp_start - upstream_len - 1: temp_start -1]
    
    elif df.loc[index]["strand"] == "-":
    
        upstream_seq = genome.seq[temp_stop : temp_stop + upstream_len]
        upstream_seq = upstream_seq.reverse_complement()
    
    upstream_seq = str(upstream_seq)
    
    df.at[index, "upstream_sequence"] = upstream_seq
    

In [ ]:
for index in df.index:
    if len(df.at[index, "coding_sequence"])%3 != 0 : 
        print(index)

In [ ]:
df["coding_sequence"]

# Testing Library

In [2]:
import gff3_parsing

from gff3_parsing import compile_sequences

In [3]:
new_df, new_genome = gff3_parsing.compile_sequences("../Data/examples_for_testing/ecoli.gff3",\
                                                    "../Data/examples_for_testing/ecoli.fasta", 20)

In [4]:
new_df.head(10)

,genome_id,source,type,start,stop,idk,strand,trash,qualifiers,coding_sequence,upstream_sequence
2,NC_000913.3,RefSeq,CDS,190,255,.,+,0,ID=cds-NP_414542.1;Parent=gene-b0001;Dbxref=Un...,ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCA...,TTACAGAGTACACAACATCC
4,NC_000913.3,RefSeq,CDS,337,2799,.,+,0,ID=cds-NP_414543.1;Parent=gene-b0002;Dbxref=Un...,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,AAGGTAACGAGGTAACAACC
6,NC_000913.3,RefSeq,CDS,2801,3733,.,+,0,ID=cds-NP_414544.1;Parent=gene-b0003;Dbxref=Un...,ATGGTTAAAGTTTATGCCCCGGCTTCCAGTGCCAATATGAGCGTCG...,ATGGAAGTTAGGAGTCTGAC
8,NC_000913.3,RefSeq,CDS,3734,5020,.,+,0,ID=cds-NP_414545.1;Parent=gene-b0004;Dbxref=Un...,ATGAAACTCTACAATCTGAAAGATCACAACGAGCAGGTCAGCTTTG...,CACGAGTACTGGAAAACTAA
10,NC_000913.3,RefSeq,CDS,5234,5530,.,+,0,ID=cds-NP_414546.1;Parent=gene-b0005;Dbxref=Un...,GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGG...,AATGATAAAAGGAGTAACCT
14,NC_000913.3,RefSeq,CDS,5683,6459,.,-,0,ID=cds-NP_414547.1;Parent=gene-b0006;Dbxref=Un...,ATGCTGATTCTTATTTCACCTGCGAAAACGCTTGATTACCAAAGCC...,ATTTCCTGCAAGGACTGGAT
16,NC_000913.3,RefSeq,CDS,6529,7959,.,-,0,ID=cds-NP_414548.1;Parent=gene-b0007;Dbxref=Un...,ATGCCAGATTTTTTCTCCTTCATTAACAGCGTCCTTTGGGGATCGG...,CGGCAATAAGAGGGATATGC
18,NC_000913.3,RefSeq,CDS,8238,9191,.,+,0,ID=cds-NP_414549.1;Parent=gene-b0008;Dbxref=Un...,ATGACGGACAAATTGACCTCCCTTCGTCAGTACACCACCGTAGTGG...,GTTTAAAGAGAAATACTATC
20,NC_000913.3,RefSeq,CDS,9306,9893,.,+,0,ID=cds-NP_414550.1;Parent=gene-b0009;Dbxref=Un...,ATGAATACTTTACGTATTGGCTTAGTTTCCATCTCTGATCGCGCAT...,GTTTAAACGGAAAAATCTTG
22,NC_000913.3,RefSeq,CDS,9928,10494,.,-,0,ID=cds-NP_414551.1;Parent=gene-b0010;Dbxref=Un...,ATGGGCAACACTAAGTTGGCTAATCCGGCACCGCTGGGCCTGATGG...,ATGATGATTTTTGAGGAATT
